# Lab 6: Saving and Sending Navigation Goals
**Name:** Alan Nur 
**Date:** February 17, 2026

## 1. Introduction
In this lab, I developed a system to programmatically save and recall robot positions on a map. The goal was to create a tool where a user can drive a robot to a specific spot, name that spot, save it to a database, and later command the robot to return to that exact location autonomously. This mimics real-world applications where robots need to remember semantic locations like "kitchen" or "charging station."

## 2. System Architecture
I created a ROS 2 package named `lab6` containing two custom Python nodes:

- **Map Annotator (`map_annotator.py`):** This is the main logic node. It listens to the robot's current position, processes user commands from the terminal, saves data to a CSV file using pandas, and publishes navigation goals.

- **UI Markers (`ui_markers.py`):** This node visualizes the saved locations in Rviz2. It reads the list of saved poses and publishes Arrow markers so the user can see where the goals are located on the map.

## 3. Implementation

### Part A: The Map Annotator Node
The `map_annotator` node handles the terminal interface. It runs a separate thread for input to ensure it does not block ROS callbacks.

**Key features:**

*   **Subscribers:** Listens to `/amcl_pose` to know where the robot is.
*   **Publishers:** Sends goals to `/goal_pose` and the list of saved spots to `/map_poses`.
*   **Data Persistence:** When the user types `exit`, the node saves the dictionary of poses to a `.csv` file. On startup, it reloads them.

Here is the implementation for the annotator:

In [ ]:
# map_annotator.py
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import PoseWithCovarianceStamped, PoseStamped, PoseArray, Pose
import pandas as pd
import threading
import sys
import os

class MapAnnotator(Node):
    def __init__(self):
        super().__init__('map_annotator')
        
        # Subscriber untuk tahu posisi robot saat ini (biasanya dari AMCL)
        self.pose_sub = self.create_subscription(
            PoseWithCovarianceStamped,
            '/amcl_pose',
            self.pose_callback,
            10)
        
        # Publisher untuk mengirim perintah jalan (goto)
        self.goal_pub = self.create_publisher(PoseStamped, '/goal_pose', 10)
        
        # Publisher untuk mengirim daftar pose ke UI Marker
        self.pose_array_pub = self.create_publisher(PoseArray, '/map_poses', 10)

        self.current_pose = None
        self.poses_dict = {} # Menyimpan data pose di memori
        
        # Setup file database
        # Kita pakai absolute path agar aman
        self.file_path = os.path.expanduser('~/turtlebot4_ws/src/lab6/poses.csv')
        self.load_poses()

        # Timer untuk terus publish PoseArray agar marker muncul di Rviz
        self.create_timer(1.0, self.publish_pose_list)

        print("Map Annotator Started.")
        self.print_menu()

    def pose_callback(self, msg):
        # Simpan posisi robot terakhir yang diketahui
        self.current_pose = msg.pose.pose

    def load_poses(self):
        # Load data dari CSV jika ada
        if os.path.exists(self.file_path):
            try:
                df = pd.read_csv(self.file_path)
                for index, row in df.iterrows():
                    p = Pose()
                    p.position.x = row['px']
                    p.position.y = row['py']
                    p.position.z = row['pz']
                    p.orientation.x = row['ox']
                    p.orientation.y = row['oy']
                    p.orientation.z = row['oz']
                    p.orientation.w = row['ow']
                    self.poses_dict[row['name']] = p
                print(f"Loaded {len(self.poses_dict)} poses from database.")
            except Exception as e:
                print(f"Error loading CSV: {e}")

    def save_poses_to_file(self):
        # Simpan data ke CSV saat exit
        data = []
        for name, p in self.poses_dict.items():
            data.append({
                'name': name,
                'px': p.position.x, 'py': p.position.y, 'pz': p.position.z,
                'ox': p.orientation.x, 'oy': p.orientation.y, 
                'oz': p.orientation.z, 'ow': p.orientation.w
            })
        
        df = pd.DataFrame(data)
        # Format kolom sesuai permintaan
        cols = ['name', 'px', 'py', 'pz', 'ox', 'oy', 'oz', 'ow']
        df = df[cols] if not df.empty else pd.DataFrame(columns=cols)
        
        df.to_csv(self.file_path, index=False)
        print(f"Poses saved to {self.file_path}")

    def publish_pose_list(self):
        # Publish PoseArray untuk node ui_markers
        msg = PoseArray()
        msg.header.frame_id = 'map'
        msg.header.stamp = self.get_clock().now().to_msg()
        
        for p in self.poses_dict.values():
            msg.poses.append(p)
        
        self.pose_array_pub.publish(msg)

    def print_menu(self):
        print("\ncommands:")
        print(" list")
        print(" save <name>")
        print(" delete <name>")
        print(" goto <name>")
        print(" help")
        print(" exit")

    def handle_input(self):
        # Loop ini jalan di thread terpisah agar tidak memblokir ROS
        while rclpy.ok():
            try:
                # Pakai input() biasa, command prompt tanda '>'
                cmd_str = input("> ").strip()
                parts = cmd_str.split()
                if not parts:
                    continue

                cmd = parts[0]
                arg = parts[1] if len(parts) > 1 else None

                if cmd == 'list':
                    print("Poses:")
                    if not self.poses_dict:
                        print(" No poses")
                    for name in self.poses_dict:
                        print(f" {name}")

                elif cmd == 'save':
                    if not arg:
                        print("Error: usage 'save <name>'")
                        continue
                    if self.current_pose is None:
                        print("Error: No pose received from robot yet.")
                        continue
                    
                    # Simpan pose saat ini
                    self.poses_dict[arg] = self.current_pose
                    print(f"Saved pose '{arg}'")

                elif cmd == 'delete':
                    if not arg:
                        print("Error: usage 'delete <name>'")
                        continue
                    if arg in self.poses_dict:
                        del self.poses_dict[arg]
                        print(f"Deleted '{arg}'")
                    else:
                        print(f"'{arg}' does not exist")

                elif cmd == 'goto':
                    if not arg:
                        print("Error: usage 'goto <name>'")
                        continue
                    if arg in self.poses_dict:
                        target_pose = self.poses_dict[arg]
                        
                        # Buat pesan PoseStamped
                        goal = PoseStamped()
                        goal.header.frame_id = 'map'
                        goal.header.stamp = self.get_clock().now().to_msg()
                        goal.pose = target_pose
                        
                        self.goal_pub.publish(goal)
                        print(f"Sent goal to '{arg}'")
                    else:
                        print(f"'{arg}' does not exist")

                elif cmd == 'help':
                    self.print_menu()

                elif cmd == 'exit':
                    self.save_poses_to_file()
                    print("Exiting...")
                    rclpy.shutdown()
                    break
                
                else:
                    print("Unknown command. Type 'help'.")

            except EOFError:
                break
            except Exception as e:
                print(f"Error processing input: {e}")

def main(args=None):
    rclpy.init(args=args)
    node = MapAnnotator()
    
    # Jalankan input user di thread terpisah
    input_thread = threading.Thread(target=node.handle_input)
    input_thread.start()
    
    # Jalankan node ROS di main thread
    try:
        rclpy.spin(node)
    except KeyboardInterrupt:
        pass
    finally:
        if rclpy.ok():
            node.save_poses_to_file()
            rclpy.shutdown()
        input_thread.join()

if __name__ == '__main__':
    main()

### Part B: The UI Markers Node
This node provides visual feedback. It subscribes to the `/map_poses` topic published by the annotator. It iterates through the list of poses and creates a `MarkerArray` message. I used Arrow markers (type 0) to clearly show both the position (x, y) and the orientation (where the robot is facing).

Here is the implementation for the markers:

In [ ]:
# ui_markers.py
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import PoseArray
from visualization_msgs.msg import Marker, MarkerArray

class UiMarkers(Node):
    def __init__(self):
        super().__init__('ui_markers')
        
        # Subscribe ke topic yang dipublish map_annotator
        self.sub = self.create_subscription(
            PoseArray,
            '/map_poses',
            self.callback,
            10)
        
        # Publish MarkerArray ke Rviz
        self.marker_pub = self.create_publisher(MarkerArray, '/visualization_marker_array', 10)
        print("UI Markers Node Started.")

    def callback(self, msg):
        marker_array = MarkerArray()
        
        # Loop setiap pose dan buat markernya
        # Kita pakai 'Arrow' agar arah (orientasi) robot terlihat jelas
        for i, pose in enumerate(msg.poses):
            marker = Marker()
            marker.header = msg.header
            marker.ns = "saved_poses"
            marker.id = i
            marker.type = Marker.ARROW
            marker.action = Marker.ADD
            
            marker.pose = pose
            
            # Ukuran marker (panjang 0.5m)
            marker.scale.x = 0.5
            marker.scale.y = 0.05
            marker.scale.z = 0.05
            
            # Warna marker (misal: Hijau)
            marker.color.a = 1.0 # Alpha harus 1.0 agar terlihat
            marker.color.r = 0.0
            marker.color.g = 1.0
            marker.color.b = 0.0
            
            marker_array.markers.append(marker)
            
            # Opsional: Tambahkan TEXT_VIEW_FACING untuk nama (kalau data nama dikirim)
            # Karena PoseArray standar tidak bawa nama, kita pakai Arrow saja dulu.
            
        self.marker_pub.publish(marker_array)

def main(args=None):
    rclpy.init(args=args)
    node = UiMarkers()
    rclpy.spin(node)
    rclpy.shutdown()

if __name__ == '__main__':
    main()

## 4. Challenges and Solutions
During the development process, I encountered and solved a few key issues:

- **Rviz Topic Mismatch:** Initially, the markers did not appear in Rviz even though the node was running. I discovered that the MarkerArray display in Rviz was listening to a default topic named `waypoints`, while my node was publishing to `/visualization_marker_array`. Changing the topic setting in Rviz fixed the visualization.

- **File Paths:** To ensure the CSV file could be saved and loaded correctly regardless of where I ran the command, I used absolute paths in the Python script.

- **Navigation Readiness:** I learned that the robot must be localized first using "2D Pose Estimate" in Rviz before the `map_annotator` can successfully save a pose. If AMCL does not know the robot's location, the save command returns an error.

## 5. Deliverables

### Video Demonstration
Below is a link to the video demonstrating the system in action. The video shows:

1. Saving multiple poses (e.g., `test_kanan`, `test_kiri`).
2. Visualizing them as green arrows in Rviz.
3. Commanding the robot to `goto` a saved location.
4. Persisting data by restarting the node and listing the saved poses.

<iframe width="560" height="315" src="https://www.youtube.com/embed/mczBmpeVkOU" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

### CSV Database
Below is the content of the `poses.csv` file generated during the demo. It follows the required format: `name, px, py, pz, ox, oy, oz, ow`.

In [ ]:
# poses.csv
name,px,py,pz,ox,oy,oz,ow
loc_1,1.6901639859373414,-1.8117276774333213,0.0,0.0,0.0,-0.8005578089518102,0.5992555335800221
loc_2,4.613703507840109,-0.8098211013123321,0.0,0.0,0.0,0.6598350536234378,0.7514104750465986

## 6. Conclusion
This lab successfully demonstrated how to interface with the Navigation2 stack using custom Python nodes. The system allows for persistent storage of semantic locations and provides a visual interface for the user. The combination of pandas for data management and Rviz markers for visualization created a functional tool for managing robot navigation goals.